In [ ]:
from dubins import connect
from propagate import *


import numpy as np
import matplotlib.pyplot as plt
from plot import plot2dcov
from nodes_edges import Edge, BeliefNode
%matplotlib notebook

def propagate(edge, start_belief, dt):
    Sigma = start_belief.Sigma
    states = edge.states
    actions = edge.actions
    state = states[0]
    Lambda = start_belief.Lambda
    xy_error = 1e-6
    theta_error = 1e-6
    measurement_error = 1e-6
    k = 1e-9
    observable_check_const = 1

    kalman = DynamicModel(xy_error, theta_error,measurement_error, dt,k)
#     print(states)
    for action in actions:
        print(state,action)
        kalman.predict(state, action, Sigma)
        kalman.update(Lambda)
        state = kalman.x_updated
        Sigma = kalman.Sigma_updated
        Lambda = kalman.Lambda
    
    print(len(actions))
#     print(states[0])
#     print(state,states[-1])

    return BeliefNode(Sigma, Lambda, len(actions))    

In [ ]:
start = np.array([250.0, 50.0, 0.0])
end = np.array([280.0, 50.0, 0.0])
# dt = 1e-3

dt = 1e-3
r = 1.5
edge = connect(start, end,r, dt, plot = True)
edge_test = Edge(edge[0], edge[1])
belief_test = BeliefNode(np.eye(3)*0.001,np.zeros((3,3)), 0)
belief_output = propagate(edge_test, belief_test, dt)
# print(belief_output.Lambda )
# plot2dcov(end[:2], (belief_output.Lambda)[:2,:2])

In [ ]:
import numpy as np

def AppendBelief(v, new_belief, eps):
    dominated = True
    for node in v.nodes:
        if np.linalg.norm(new_belief.Sigma) < np.linalg.norm(node.Sigma+np.eye(3)*eps) and
        np.linalg.norm(new_belief.Lambda) < np.linalg.norm(node.Lambda+np.eye(3)*eps) and
        new_belief.cost < node.cost:
            dominated = False
    if dominated:
        return False
    else:
        to_delete = []
        for node in v.nodes.copy():
            if (np.linalg.norm(new_belief.Sigma) < np.linalg.norm(node.Sigma) and
            np.linalg.norm(new_belief.Lambda) < np.linalg.norm(node.Lambda) and
            new_belief.cost < node.cost_:
                v.nodes.remove(node)
    v.nodes.append(new_belief)